# GroupfMRI6a - Use motor rule regions to predict Motor response regions
#### Using ActFlow

#### Takuya Ito
#### 03/01/2019

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import multiprocessing as mp
import scipy.stats as stats
import os
os.environ['OMP_NUM_THREADS'] = str(1)
import statsmodels.sandbox.stats.multicomp as mc
import seaborn as sns
import h5py
import tools_group
import nibabel as nib
import EmpiricalSRActFlow as esr
sns.set_style("whitegrid")
plt.rcParams["font.family"] = "FreeSans"


In [2]:
# Excluding 084
subjNums = ['013','014','016','017','018','021','023','024','026','027','028','030','031','032','033',
            '034','035','037','038','039','040','041','042','043','045','046','047','048','049','050',
            '053','055','056','057','058','062','063','066','067','068','069','070','072','074','075',
            '076','077','081','085','086','087','088','090','092','093','094','095','097','098','099',
            '101','102','103','104','105','106','108','109','110','111','112','114','115','117','119',
            '120','121','122','123','124','125','126','127','128','129','130','131','132','134','135',
            '136','137','138','139','140','141']



basedir = '/projects3/SRActFlow/'

# Using final partition
networkdef = np.loadtxt('/projects3/NetworkDiversity/data/network_partition.txt')
networkorder = np.asarray(sorted(range(len(networkdef)), key=lambda k: networkdef[k]))
networkorder.shape = (len(networkorder),1)
# network mappings for final partition set
networkmappings = {'fpn':7, 'vis1':1, 'vis2':2, 'smn':3, 'aud':8, 'lan':6, 'dan':5, 'con':4, 'dmn':9, 
                   'pmulti':10, 'none1':11, 'none2':12}
networks = networkmappings.keys()

xticks = {}
reorderednetworkaffil = networkdef[networkorder]
for net in networks:
    netNum = networkmappings[net]
    netind = np.where(reorderednetworkaffil==netNum)[0]
    tick = np.max(netind)
    xticks[tick] = net

## General parameters/variables
nParcels = 360
nSubjs = len(subjNums)

glasserfile2 = '/projects/AnalysisTools/ParcelsGlasser2016/Q1-Q6_RelatedParcellation210.LR.CorticalAreas_dil_Colors.32k_fs_RL.dlabel.nii'
glasser2 = nib.load(glasserfile2).get_data()
glasser2 = np.squeeze(glasser2)

sortednets = np.sort(xticks.keys())
orderednetworks = []
for net in sortednets: orderednetworks.append(xticks[net])
    
networkpalette = ['royalblue','slateblue','paleturquoise','darkorchid','limegreen',
                  'lightseagreen','yellow','orchid','r','peru','orange','olivedrab']
networkpalette = np.asarray(networkpalette)

OrderedNetworks = ['VIS1','VIS2','SMN','CON','DAN','LAN','FPN','AUD','DMN','PMM','VMM','ORA']

# 1.0 Define functions for motor response decodings

#### Load in FC mappings

In [3]:
# import GroupfMRI5a_ComputePCA_FC as tmp
# tmp = reload(tmp)
# inputtypes = ['ori','color','pitch','constant']
# for inputtype in inputtypes:
#     print 'Computing group fc for inputtype', inputtype
#     tmp.computeGroupFCMappings(inputtype)

In [4]:
esr = reload(esr)
tools_group = reload(tools_group)

def subjSRActFlow(subj):
    inputtypes = ['color','ori','pitch','constant']
    inputkeys = ['RED','VERTICAL','HIGH','CONSTANT']
    obj = esr.EmpiricalActFlow(subj)
    
    # Load FC for SRActFlow
    fc_input2sr = {}
    fc_sr2motorrule = {}
    i = 0
    for inputtype in inputtypes:
        fc_input2sr[inputkeys[i]], fc_sr2motorrule[inputkeys[i]], fc_motorrule2motorresp = tools_group.loadSubjActFlowFC(subj,inputtype)
        i += 1

    # Save to obj 
    obj.fc_input2sr = fc_input2sr
    obj.fc_sr2motorrule = fc_sr2motorrule
    obj.fc_motorrule2resp = fc_motorrule2motorresp

    actflow = obj.generateActFlowPredictions(verbose=False, n_inputs=2)
    del obj
    return actflow

    
pool = mp.Pool(processes=10)
results = pool.map_async(subjSRActFlow,subjNums).get()
pool.close()
pool.join()

actflow_predictions = np.asarray(results)

EmpiricalSRActFlow.py:182: ComplexWarning: Casting complex values to real discards the imaginary part
  actflow[:,0,nstim] = np.mean(actflow_predictions[ind,:],axis=0)
EmpiricalSRActFlow.py:185: ComplexWarning: Casting complex values to real discards the imaginary part
  actflow[:,1,nstim] = np.mean(actflow_predictions[ind,:],axis=0)
EmpiricalSRActFlow.py:188: ComplexWarning: Casting complex values to real discards the imaginary part
  actflow[:,2,nstim] = np.mean(actflow_predictions[ind,:],axis=0)
EmpiricalSRActFlow.py:191: ComplexWarning: Casting complex values to real discards the imaginary part
  actflow[:,3,nstim] = np.mean(actflow_predictions[ind,:],axis=0)
EmpiricalSRActFlow.py:182: ComplexWarning: Casting complex values to real discards the imaginary part
  actflow[:,0,nstim] = np.mean(actflow_predictions[ind,:],axis=0)
EmpiricalSRActFlow.py:185: ComplexWarning: Casting complex values to real discards the imaginary part
  actflow[:,1,nstim] = np.mean(actflow_predictions[ind,:],

## Load real motor response data

In [5]:
tools_group = reload(tools_group)
nResponses = 2
data_task_rh = np.zeros((len(glasser2),nResponses,len(subjNums)))
data_task_lh = np.zeros((len(glasser2),nResponses,len(subjNums)))

scount = 0
for subj in subjNums:
    data_task_rh[:,:,scount] = tools_group.loadMotorResponses(subj,hand='Right')
    data_task_lh[:,:,scount] = tools_group.loadMotorResponses(subj,hand='Left')
    scount += 1

tools_group.py:95: ComplexWarning: Casting complex values to real discards the imaginary part
  tmpdat[:,0] = data[:,2]
tools_group.py:96: ComplexWarning: Casting complex values to real discards the imaginary part
  tmpdat[:,1] = data[:,3]
tools_group.py:98: ComplexWarning: Casting complex values to real discards the imaginary part
  tmpdat[:,0] = data[:,0] #lmid
tools_group.py:99: ComplexWarning: Casting complex values to real discards the imaginary part
  tmpdat[:,1] = data[:,1] #lind


In [6]:
# Set indices for layer-by-layer vertices
targetdir = '/projects3/SRActFlow/data/results/GroupfMRI/MotorResponseDecoding/'
motor_resp_regions_LH = np.loadtxt(targetdir + 'MotorResponseRegions_LH.csv',delimiter=',')
motor_resp_regions_RH = np.loadtxt(targetdir + 'MotorResponseRegions_RH.csv',delimiter=',')
targetROIs = np.hstack((motor_resp_regions_LH,motor_resp_regions_RH))

target_ind = []
for roi in targetROIs:
    roi_ind = np.where(glasser2==roi+1)[0]
    target_ind.extend(roi_ind)
target_ind = np.asarray(target_ind)

## 1.1 Run across subject decoding on right-hand motor responses

### Compute average activity for each response, for each subject

In [9]:
scount = 0
actflow_rh = np.zeros(data_task_rh.shape)
actflow_lh = np.zeros(data_task_lh.shape)
for scount in range(len(subjNums)):
    # RMID
    actflow_rh[target_ind,0,scount] = np.mean(actflow_predictions[scount,:,2,:],axis=1)
    # RIND
    actflow_rh[target_ind,1,scount] = np.mean(actflow_predictions[scount,:,3,:],axis=1)
    # LMID
    actflow_lh[target_ind,0,scount] = np.mean(actflow_predictions[scount,:,0,:],axis=1)
    # LIND
    actflow_lh[target_ind,1,scount] = np.mean(actflow_predictions[scount,:,1,:],axis=1)


In [164]:
tools_group = reload(tools_group)
nproc = 20
nResponses = 2
ncvs = 1

rois = np.asarray([8,9])-1
# rois = np.asarray([9])-1
# rois = np.where(networkdef==networkmappings['smn'])[0]
roi_ind = []
for roi in rois:
    roi_ind.extend(np.where(glasser2==roi+1)[0])

# realdata = stats.zscore(data_task_rh[target_ind,:,:],axis=0).copy()
# flowdata = stats.zscore(actflow_rh[target_ind,:,:],axis=0).copy()
realdata_rh = data_task_rh[roi_ind,:,:].copy()
flowdata_rh = actflow_rh[roi_ind,:,:].copy()


distances_baseline_rh = np.zeros((1,len(subjNums)*nResponses))
distances_baseline_rh[0,:],rmatch,rmismatch = tools_group.actflowDecodings(realdata_rh,
                                                          flowdata_rh,effects=True,
                                                          ncvs=ncvs, nproc=nproc)

## 1.2 Compute statistics

In [165]:
statistics_rh = np.zeros((distances_baseline_rh.shape[0],2))
ntrials = len(subjNums)*2
p = stats.binom_test(np.mean(distances_baseline_rh[0,:])*ntrials,n=ntrials,p=0.5)
if np.mean(distances_baseline_rh[0,:])>0.5:
    p = p/2.0
else:
    p = 1.0-p/2.0

    
statistics_rh[0,0] = np.mean(distances_baseline_rh[0,:])
statistics_rh[0,1] = p

print 'Activity flow accuracy =', statistics_rh[0,0]
print 'p =', statistics_rh[0,1]
print 'Matched spatial correlation:', np.mean(rmatch)
print 'Mismatched spatial correlation:', np.mean(rmismatch)



Activity flow accuracy = 0.546875
p = 0.1098858055254432
Matched spatial correlation: 0.0035166213725783824
Mismatched spatial correlation: -0.0013453996993430032


# 2.0 Run across subject decoding on left-hand motor responses

In [160]:
tools_group = reload(tools_group)
nproc = 20
ncvs = 1

# rois = np.asarray([188,189]) - 1
rois = np.asarray([189]) - 1
# rois = np.where(networkdef==networkmappings['smn'])[0]
roi_ind = []
for roi in rois:
    roi_ind.extend(np.where(glasser2==roi+1)[0])
    
# realdata = stats.zscore(data_task_lh[roi_ind,:,:],axis=0).copy()
# flowdata = stats.zscore(actflow_lh[roi_ind,:,:],axis=0).copy()
realdata_lh = data_task_lh[roi_ind,:,:].copy()
flowdata_lh = actflow_lh[roi_ind,:,:].copy()


distances_baseline_lh = np.zeros((1,len(subjNums)*nResponses))
distances_baseline_lh[0,:],rmatch,rmismatch = tools_group.actflowDecodings(realdata_lh,
                                                          flowdata_lh,effects=True,
                                                          ncvs=ncvs, nproc=nproc)
    

## 1.2 Compute statistics

In [161]:
statistics_lh = np.zeros((distances_baseline_lh.shape[0],2))
ntrials = len(subjNums)*2
p = stats.binom_test(np.mean(distances_baseline_lh[0,:])*ntrials,n=ntrials,p=0.5)
if np.mean(distances_baseline_lh[0,:])>0.5:
    p = p/2.0
else:
    p = 1.0-p/2.0

    
statistics_lh[0,0] = np.mean(distances_baseline_lh[0,:])
statistics_lh[0,1] = p

print 'Activity flow accuracy =', statistics_lh[0,0]
print 'p =', statistics_lh[0,1]
print 'Matched spatial correlation:', np.mean(rmatch)
print 'Mismatched spatial correlation:', np.mean(rmismatch)



Activity flow accuracy = 0.4270833333333333
p = 0.974467622660913
Matched spatial correlation: 0.00022677232839241014
Mismatched spatial correlation: 0.0018400079827300255
